In [2]:
your_username = 'somayeh91'


# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
import sys

# sys.path.append('/home/idies/Storage/'+your_usernamer+'/persistent/sims_maf_contrib/')
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
print(os.getcwd())
# import convenience functions
from opsimUtils import *

dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.5/'
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/db_results_11_15_2020/'.format(your_username)

resultsDb = db.ResultsDb(outDir=outDir)
if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))
    
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

/home/idies/workspace/Storage/somayeh91/persistent/Rubin_Microlensing_metrics


In [7]:
# !pip install sympy

In [3]:
import glob 

dbruns_selected = glob.glob(dbDir+"footprint*")
dbruns_selected.sort()

In [14]:
dbruns_selected

['/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_add_mag_cloudsv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_big_sky_dustv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_big_sky_nouiyv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_big_skyv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_big_wfdv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_bluer_footprintv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_gp_smoothv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_newAv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_newBv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_no_gp_northv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_standard_goalsv1.5_10yrs.db',
 '/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_stuck_rollingv1.5_10yrs.db']

In [5]:
# pip install sympy

In [6]:
# WFIRST_OW = [['2025-04-25', '2025-07-07'],['2026-04-25', '2026-07-07'],['2025-04-25', '2026-07-07']]

In [10]:
LSST_filters = ["g","i","z"]#,"g","r","i","z"]
import sys
sys.path.insert(0, '../sims_maf_contrib/')

summaryMetrics = [metrics.MedianMetric(), metrics.MeanMetric(), metrics.MinMetric(), metrics.MaxMetric(), metrics.RmsMetric(),\
                 metrics.SumMetric(), metrics.CountMetric()]

from mafContrib import StarCountMetric,CampaignLengthMetric, numObsInSurveyTimeOverlapMetric, IntervalsBetweenObsMetric
# sys.path.insert(0, '../sims_maf_contrib/mafContrib')
# from numObsInSurveyTimeOverlapMetric import numObsInSurveyTimeOverlapMetric
starcount= StarCountMetric()
campaign =  CampaignLengthMetric()
# numobs = numObsInSurveyTimeOverlapMetric(WFIRST_OW)
# intervalObs = IntervalsBetweenObsMetric(WFIRST_OW, Stat = 'median')
cadence = metrics.cadenceMetrics.NRevisitsMetric()
uniformity = metrics.UniformityMetric()
#List of metrics

Microlensing_metrics = [ starcount, campaign, cadence, uniformity]

names_metric = ['starcount','campaignlength','nrevisits', 'uniformity']
# names_metric = ['campaignlength','nrevisits', 'uniformity']



In [11]:
#Slicer
slicer1 = slicers.HealpixSlicer(nside=64)


#List of bundles
Microlensing_bundles = []
Names_metric = []
for ind_fil,fil in enumerate(LSST_filters):
    
    if fil == 'u':
        continue
    
    constraint1 = 'filter = "'+fil+'"'
    print(constraint1)
    for ind_met,metri in enumerate(Microlensing_metrics):
        
        microlensing_metric = metricBundles.MetricBundle(metri, slicer1, constraint1, summaryMetrics=summaryMetrics)
        Microlensing_bundles.append(microlensing_metric)
        Names_metric.append(names_metric[ind_met]+'_'+fil)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
filter = "g"
filter = "i"
filter = "z"


In [15]:

# create a bundleDict for all metrics to run on each opSim
bundleDict = {Names_metric[i]:Microlensing_bundles[i] for i in range(len(Names_metric))}
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/metric_data_11_15_2020/'.format(your_username)

In [ ]:
# metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/output_10_26_2020/'.format(your_username)

# below I am only going to run metrics on the first five opsims
for run in dbruns_selected[5:]:
    print (run)
    run_name = run.split('1.5/')[1].split('.d')[0]
    
    # must set run name for each opSim to store metric data into
    # separate files
#     numobs.setRunName(run_name)
#     intervalObs.setRunName(run_name)
#     starcount.setRunName(run_name)
#     campaignlength.setRunName(run_name)
#     nrevisits.setRunName(run_name)
#     uniformity.setRunName(run_name)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run_name], metricDataPath, resultDbs[run_name])
    metricGroup.runAll()

/home/idies/workspace/lsst_cadence/FBS_1.5/footprint_bluer_footprintv1.5_10yrs.db
Querying database SummaryAllProps with constraint filter = "z" for columns ['fieldRA', 'fieldDec', 'observationStartMJD']
Found 352241 visits
Running:  ['starcount_z', 'campaignlength_z', 'nrevisits_z', 'uniformity_z']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "i" for columns ['fieldRA', 'fieldDec', 'observationStartMJD']
Found 483713 visits
Running:  ['starcount_i', 'campaignlength_i', 'nrevisits_i', 'uniformity_i']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" for columns ['fieldRA', 'fieldDec', 'observationStartMJD']
Found 400012 visits
Running:  ['starcount_g', 'campaignlength_g', 'nrevisits_g', 'uniformity_g']
Completed metric generation.
Running reduce methods.
Running summary statisti